In [1]:
# setting packages

using Pkg

Pkg.add(["CSV", "DataFrames", "Turing","Logging", "StatisticalRethinking", "StatisticalRethinkingPlots", "StatsBase", "Random", "LaTeXStrings", "StatsPlots", "Dagitty"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## import packages

In [2]:
using CSV
using DataFrames
using Turing
using Logging
using StatisticalRethinking
using StatisticalRethinking: link  # import explicitly, because Turing has link method also
using StatisticalRethinkingPlots
using StatsBase
using Random
using LaTeXStrings
using StatsPlots

using Dagitty

default(label=false)
Logging.disable_logging(Logging.Warn);

In [6]:
url = "https://raw.githubusercontent.com/StatisticalRethinkingJulia/SR2TuringJupyter.jl/main/data/WaffleDivorce.csv"

data = DataFrame(CSV.File(download(url)))
data[!,:D] = standardize(ZScoreTransform, data.Divorce)
data[!,:M] = standardize(ZScoreTransform, data.Marriage)
data[!,:A] = standardize(ZScoreTransform, data.MedianAgeMarriage);

In [16]:
# how to set numeric format?

println("standard deviation of median age is $(std(data.MedianAgeMarriage))")

standard deviation of median age is 1.2436303013880823


In [20]:
Random.seed!(100)

@model function model_m5_1(A, D)
    sigma ~ Exponential(1)  # how to change unicode \sigma?
    a ~ Normal(0, 0.2)
    bA ~ Normal(0, 0.5)
    mu = @. a + bA * A  # what is @ mean?
    D ~ MvNormal(mu, sigma)
end

m5_1 = sample(model_m5_1(data.A, data.D), NUTS(), 1000)
m5_1_df = DataFrame(m5_1)
prior = sample(model_m5_1([0], [0]), Prior(), 1000)
prior_df = DataFrame(prior)

prior_df[1:5,:]

,a,bA,sigma
,Float64,Float64,Float64
1,-0.241561,0.252116,0.678091
2,0.295056,-0.238246,0.90592
3,0.0121098,0.813315,0.831374
4,0.202239,-0.0774171,0.0882492
5,-0.0300674,-0.27496,0.059543


In [ ]:
# caculate \mu for every prior sample on age=-2 and age=2
